In [21]:
import json
from pathlib import Path
from typing import Dict, Any
from bert_score import score
from langchain_community.llms import LlamaCpp
from langchain.prompts import PromptTemplate

In [6]:
def load_json_files_separately(folder_path: str) -> Dict[str, Any]:
    """
    Charge tous les fichiers JSON dans un dossier.
    Chaque fichier est retourné comme un objet Python, stocké dans un dict.

    Retourne un dict du type :
    {
        "query1": {...},
        "query2": {...},
        ...
    }
    """
    folder = Path(folder_path)
    json_objects = {}

    for json_file in folder.glob("*.json"):
        with json_file.open("r", encoding="utf-8") as f:
            json_objects[json_file.stem] = json.load(f)

    return json_objects

In [36]:
data = load_json_files_separately("../data/dataset_eval")

In [20]:
llm = LlamaCpp(
    model_path="../data/models/mistral-7b-instruct-v0.2.Q4_0.gguf",
    n_ctx=8192,
    n_batch=64,
    temperature=0.0,
    max_tokens=512,
    n_threads=6,
    verbose=False,
    use_mlock=True,
)

llama_context: n_ctx_per_seq (8192) < n_ctx_train (32768) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h64           (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h80           (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h96 

In [50]:
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="Contexte : {context}\n\nQuestion : {question}\nRéponse :"
)
responses = []
answers = []
queries_id = []

for i, query_id in enumerate(data["queries"].keys()):
    query = data["queries"][query_id]
    corpus_id = data["relevant_docs"][query_id][0]
    corpus = data["corpus"][corpus_id]
    answer = data["answers"][query_id]
    prompt_input = prompt_template.format(context=corpus, question=query)
    queries_id.append(query_id)
    responses.append(llm(prompt_input))
    answers.append(answer)

    if i % 10 == 0:
        print(i)
        response = dict(zip(queries_id, responses))
        with open("../data/dataset_eval/response_{}.json".format(i), "w", encoding="utf-8") as f:
            json.dump(response, f, indent=4, ensure_ascii=False)
    


0
10


KeyboardInterrupt: 

In [78]:
response = dict(zip(queries_id, responses))

with open("../data/dataset_eval/response.json", "w", encoding="utf-8") as f:
    json.dump(response, f, indent=4, ensure_ascii=False)

In [79]:
with open("../data/dataset_eval/response.json", "r", encoding="utf-8") as f:
    response = json.load(f)

In [80]:
responses = []
answers = []
for query_id in response.keys():
    answers.append(data["answers"][query_id])
    responses.append(response[query_id])

In [81]:
P, R, F1 = score(responses, answers, lang="fr", verbose=True)

calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:04<00:00,  4.41s/it]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 56.24it/s]

done in 4.43 seconds, 3.83 sentences/sec


In [82]:
P, R, F1

(tensor([0.9066, 0.7511, 0.7684, 0.5396, 0.8400, 0.5889, 0.5316, 0.8691, 0.8226,
         0.9474, 0.8038, 0.9281, 0.8249, 0.8842, 0.6675, 0.5374, 0.6390]),
 tensor([0.9625, 0.8789, 0.9058, 0.7429, 0.9184, 0.7499, 0.7572, 0.7974, 0.8478,
         0.9652, 0.9100, 0.9483, 0.9240, 0.9209, 0.7932, 0.8368, 0.7641]),
 tensor([0.9337, 0.8100, 0.8315, 0.6252, 0.8774, 0.6597, 0.6247, 0.8317, 0.8350,
         0.9562, 0.8536, 0.9381, 0.8717, 0.9022, 0.7250, 0.6545, 0.6960]))

In [83]:
F1.mean()

tensor(0.8015)

In [ ]:
# query_id = "6ca4da60-a74c-45cb-b41c-c5fac380b7b9"
# query = data["queries"][query_id]
# corpus_id = data["relevant_docs"][query_id][0]
# corpus = data["corpus"][corpus_id]
# answer = data["answers"][query_id]

# print(query, corpus, answer, sep="\n \n")

Quel est l'objectif de gestion du FCP décrit dans le document ?
 
L’objectif de gestion du FCP est, au travers d’une gestion discrétionnaire, la recherche d’une 
performance sur la durée de placement recommandée supérieure à 5 ans par un placement sur les marchés financiers actions internationales, pays émergents inclus, tout en tenant compte des enjeux de développement durable. Les décisions d’investissement intègrent à la fois des critères financiers et extra-financiers. La prise en compte des critères de responsabilité environnementale, sociale et de gouvernance de l’entreprise (ESG) dans la sélection de titres vise à évaluer la capacité des entreprises à transformer les enjeux du développement durable en vecteurs de performance.  
A titre d’information, la performance du FCP pourra être comparée a posteriori à l’indicateur
 
La recherche d'une performance sur la durée de placement recommandée supérieure à 5 ans par un placement sur les marchés financiers actions internationales, pa